# Crunch raw CMIP6 data

Here we calculate the country-means.

In [ ]:
%load_ext nb_black

In [ ]:
import glob
import logging
import os

import iris.analysis
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import netcdf_scm
import netcdf_scm.crunching
import netcdf_scm.weights
import numpy as np
import pandas as pd
import regionmask
import xarray as xr
from netcdf_scm.iris_cube_wrappers import CMIP6OutputCube

import config

In [ ]:
ID = config.ID

In [ ]:
NETCDF_SCM_LOGGER = logging.getLogger("netcdf_scm")

In [ ]:
STDERR_INFO_HANDLER = logging.StreamHandler()
FORMATTER = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
STDERR_INFO_HANDLER.setFormatter(FORMATTER)
STDERR_INFO_HANDLER.setLevel(logging.INFO)

In [ ]:
NETCDF_SCM_LOGGER.setLevel(logging.DEBUG)
NETCDF_SCM_LOGGER.addHandler(STDERR_INFO_HANDLER)

In [ ]:
netcdf_scm.__version__

In [ ]:
!find /data/cmip6/CMIP6/ScenarioMIP -mindepth 2 -maxdepth 2 -type d -exec sh -c 'x={};echo $(basename ${x})' \; | sort

## Setup

In [ ]:
CRUNCH_DIR = "./{}-country-crunch-popn-weighted".format(ID)
CRUNCH_DIR

In [ ]:
!mkdir -p {CRUNCH_DIR}

In [ ]:
too_small_regions = {
    "World|Natural Earth 50m|San Marino",
    "World|Natural Earth 50m|Turks and Caicos Is.",
    "World|Natural Earth 50m|Indian Ocean Ter.",
    "World|Natural Earth 50m|Falkland Is.",
    "World|Natural Earth 50m|Israel",
    "World|Natural Earth 50m|Kiribati",
    "World|Natural Earth 50m|Jamaica",
    "World|Natural Earth 50m|Fr. S. Antarctic Lands",
    "World|Natural Earth 50m|Vanuatu",
    "World|Natural Earth 50m|N. Mariana Is.",
    "World|Natural Earth 50m|Palau",
    "World|Natural Earth 50m|Liechtenstein",
    "World|Natural Earth 50m|St-Barthélemy",
    "World|Natural Earth 50m|Monaco",
    "World|Natural Earth 50m|Wallis and Futuna Is.",
    "World|Natural Earth 50m|Puerto Rico",
    "World|Natural Earth 50m|Bermuda",
    "World|Natural Earth 50m|Dominica",
    "World|Natural Earth 50m|Nauru",
    "World|Natural Earth 50m|Isle of Man",
    "World|Natural Earth 50m|Rwanda",
    "World|Natural Earth 50m|Grenada",
    "World|Natural Earth 50m|Seychelles",
    "World|Natural Earth 50m|S. Geo. and the Is.",
    "World|Natural Earth 50m|Cayman Is.",
    "World|Natural Earth 50m|Sint Maarten",
    "World|Natural Earth 50m|Maldives",
    "World|Natural Earth 50m|Norfolk Island",
    "World|Natural Earth 50m|St-Martin",
    "World|Natural Earth 50m|Bahamas",
    "World|Natural Earth 50m|British Virgin Is.",
    "World|Natural Earth 50m|Comoros",
    "World|Natural Earth 50m|St. Kitts and Nevis",
    "World|Natural Earth 50m|Barbados",
    "World|Natural Earth 50m|Palestine",
    "World|Natural Earth 50m|American Samoa",
    "World|Natural Earth 50m|St. Vin. and Gren.",
    "World|Natural Earth 50m|Brunei",
    "World|Natural Earth 50m|Curaçao",
    "World|Natural Earth 50m|Guam",
    "World|Natural Earth 50m|St. Pierre and Miquelon",
    "World|Natural Earth 50m|Jersey",
    "World|Natural Earth 50m|Macao",
    "World|Natural Earth 50m|Saint Lucia",
    "World|Natural Earth 50m|Cook Is.",
    "World|Natural Earth 50m|Fr. Polynesia",
    "World|Natural Earth 50m|Faeroe Is.",
    "World|Natural Earth 50m|Luxembourg",
    "World|Natural Earth 50m|Saint Helena",
    "World|Natural Earth 50m|Åland",
    "World|Natural Earth 50m|Niue",
    "World|Natural Earth 50m|Pitcairn Is.",
    "World|Natural Earth 50m|Br. Indian Ocean Ter.",
    "World|Natural Earth 50m|Vatican",
    "World|Natural Earth 50m|Heard I. and McDonald Is.",
    "World|Natural Earth 50m|Albania",
    "World|Natural Earth 50m|Malta",
    "World|Natural Earth 50m|N. Cyprus",
    "World|Natural Earth 50m|Ashmore and Cartier Is.",
    "World|Natural Earth 50m|Trinidad and Tobago",
    "World|Natural Earth 50m|Marshall Is.",
    "World|Natural Earth 50m|Belize",
    "World|Natural Earth 50m|U.S. Virgin Is.",
    "World|Natural Earth 50m|Antigua and Barb.",
    "World|Natural Earth 50m|Kuwait",
    "World|Natural Earth 50m|Siachen Glacier",
    "World|Natural Earth 50m|Andorra",
    "World|Natural Earth 50m|Singapore",
    "World|Natural Earth 50m|Micronesia",
    "World|Natural Earth 50m|Bahrain",
    "World|Natural Earth 50m|Togo",
    "World|Natural Earth 50m|Hong Kong",
    "World|Natural Earth 50m|Anguilla",
    "World|Natural Earth 50m|Mauritius",
    "World|Natural Earth 50m|Aruba",
    "World|Natural Earth 50m|São Tomé and Principe",
    "World|Natural Earth 50m|Solomon Is.",
    "World|Natural Earth 50m|Qatar",
    "World|Natural Earth 50m|Tonga",
    "World|Natural Earth 50m|Guernsey",
    "World|Natural Earth 50m|Montserrat",
    "World|Natural Earth 50m|Haiti",
}

In [ ]:
regions = (
    set(
        [
            "World|Natural Earth 50m|{}".format(c)
            #             for c in regionmask.defined_regions.natural_earth.countries_50.names
            for c in regionmask.defined_regions.natural_earth.countries_50.names[:1]
            + [
                #                 "Australia",
                #                 "Chile",
                #                 "Russia",
                #                 "Canada",
                #                 "France",
                #                 "United States of America",
                #                 "Israel",
                "Jamaica",
                #                 "Maldives",
                #                 "Norfolk Island",
            ]
        ]
        + ["World"]
    )
    #     - too_small_regions
)
display(len(regions))
regions = ",".join(regions)
regions

## Define custom masks

In [ ]:
gpw_v4_meta = pd.read_csv("gpw_v4_netcdf_contents_rev11.csv")
gpw_v4_meta

In [ ]:
population_raster = gpw_v4_meta[
    (gpw_v4_meta["file_name"] == "gpw_v4_population_count_rev11")
    & (gpw_v4_meta["raster_description"] == "Population count for the year 2020")
].loc[:, "order"]
population_raster = int(population_raster)
population_raster

In [ ]:
population_2020 = xr.load_dataset("gpw_v4_population_count_rev11_30_min.nc")[
    "Population Count, v4.11 (2000, 2005, 2010, 2015, 2020): 30 arc-minutes"
].sel(raster=population_raster)
population_2020

In [ ]:
population_2020.sum()

In [ ]:
population_2020.plot()

In [ ]:
population_2020.sel(
    latitude=range(20, 50 + 1), longitude=range(-130, -70 + 1), method="nearest"
).plot()

In [ ]:
population_2020_iris = population_2020.drop("raster")
population_2020_iris.attrs["units"] = "1"
population_2020_iris.name = "Population"
population_2020_iris["latitude"].attrs["standard_name"] = "latitude"
population_2020_iris["longitude"].attrs["standard_name"] = "longitude"
population_2020_iris = population_2020_iris.to_iris()
population_2020_iris.coord("latitude").guess_bounds()
population_2020_iris.coord("longitude").guess_bounds()
display(population_2020_iris)
population_2020_iris.coords()

In [ ]:
def get_natural_earth_50m_scale_nearest_last_resort_region_weights(region):
    def _calculate_region_weights(weight_calculator, cube, **kwargs):
        r_stripped = region.replace("Nearest ", "")
        weights_no_pop = weight_calculator.get_weights_array_without_area_weighting(
            r_stripped
        )
        if np.equal(np.sum(weights_no_pop), 0):
            region_mask = regionmask.defined_regions.natural_earth.countries_50[
                [region.split("|")[-1]]
            ]
            assert len(region_mask.centroids) == 1
            centre = region_mask.centroids[0]
            weights_no_pop[
                cube.cube.coord("latitude").nearest_neighbour_index(centre[1]),
                cube.cube.coord("longitude").nearest_neighbour_index(centre[0]),
            ] = 1

        return weights_no_pop

    return _calculate_region_weights


def get_natural_earth_50m_scale_popn_weighted_region_weights(region):
    def _calculate_region_weights(weight_calculator, cube, **kwargs):
        weights_no_pop = weight_calculator.get_weights_array_without_area_weighting(
            region.replace("Popn weighted ", "Nearest ")
        )

        try:
            cube.cube.coord("latitude").guess_bounds()
        except ValueError:
            pass

        try:
            cube.cube.coord("longitude").guess_bounds()
        except ValueError:
            pass

        pop_regrid = population_2020_iris.regrid(
            cube.cube, iris.analysis.AreaWeighted()
        )
        out = pop_regrid * weights_no_pop
        out = out.data
        out[np.isnan(out)] = 0

        return out

    return _calculate_region_weights

In [ ]:
CRUNCH_NEAREST_REGION = True
CRUNCH_POPULATION_WEIGHTED = True

regions_incl_pop = []
for region in regions.split(","):
    if CRUNCH_NEAREST_REGION:
        region_nearest_resort = "Nearest {}".format(region)
        netcdf_scm.weights.WEIGHTS_FUNCTIONS_WITHOUT_AREA_WEIGHTING[
            region_nearest_resort
        ] = get_natural_earth_50m_scale_nearest_last_resort_region_weights(
            region_nearest_resort
        )
        regions_incl_pop.append(region_nearest_resort)

    if CRUNCH_POPULATION_WEIGHTED:
        region_incl_pop = "Popn weighted {}".format(region)
        netcdf_scm.weights.WEIGHTS_FUNCTIONS_WITHOUT_AREA_WEIGHTING[
            region_incl_pop
        ] = get_natural_earth_50m_scale_popn_weighted_region_weights(region)
        regions_incl_pop.append(region_incl_pop)

regions_incl_pop = ",".join(regions_incl_pop)
regions_incl_pop

In [ ]:
def plot_weights(weights_to_plot, constraint=None, axes=None, **kwargs):
    for i, (label, weights) in enumerate(weights_to_plot.items()):
        if axes is None:
            ax = plt.figure().add_subplot(111)
        else:
            ax = axes[i]

        weight_cube = example.cube.collapsed("time", iris.analysis.MEAN)
        weight_cube.data = weights
        weight_cube.data[np.where(np.equal(weight_cube.data, 0))] = np.nan
        weight_cube.units = ""
        if constraint is not None:
            weight_cube = weight_cube.extract(constraint)

        plt.sca(ax)

        qplt.pcolormesh(
            weight_cube,
            **kwargs,
        )

        plt.gca().set_title(label)
        plt.gca().coastlines(alpha=0.5)

In [ ]:
example = CMIP6OutputCube()
example.load_data_in_directory(
    "/data/cmip6/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp245/r3i1p1f1/Amon/tas/gn/v20191203"
)

In [ ]:
custom_weights = example.get_scm_timeseries_weights(regions=regions_incl_pop.split(","))
plot_weights(custom_weights)

## Run

In [ ]:
scenarios = ["ssp", "historical", "piControl"]
mips = ["CMIP", "ScenarioMIP"]
scenarios = [
    "ssp119",
    "ssp126",
    "ssp245",
    "ssp370",
    "ssp370-lowNTCF",
    "ssp434",
    "ssp460",
    "ssp585",
    "ssp534-over",
    "historical",
]
members = [
    "r1i1p1f1",
    "r2i1p1f1",
    "r3i1p1f1",
    "r1i1p1f2",
    "r1i1p2f1",
    "r1i1p1f3",
    "r4i1p1f1",
    "r10i1p1f1",
    "r11i1p1f1",
]
variables = ["tas"]
tables = ["Amon"]


def get_regexp(inl, trail_slash=True):
    if trail_slash:
        return "({})".format(
            "|".join(["{}{}{}".format(os.sep, s, os.sep) for s in inl])
        )

    return "({})".format("|".join(["{}{}".format(os.sep, s) for s in inl]))


mip_regexp = get_regexp(mips)
scenarios_regexp = get_regexp(scenarios, trail_slash=False)
members_regexp = get_regexp(members, trail_slash=False)
variables_regexp = get_regexp(variables)
tables_regexp = get_regexp(tables, trail_slash=False)

regexp = ".*{}.*{}.*{}.*{}.*{}.*".format(
    mip_regexp, scenarios_regexp, members_regexp, tables_regexp, variables_regexp
)
display(regexp)

netcdf_scm.crunching._crunch_data(
    "/data/cmip6/CMIP6",
    CRUNCH_DIR,
    "Zebedee Nicholls <zebedee.nicholls@climate-energy-college.org>",
    drs="CMIP6Output",
    regexp=regexp,
    regions=regions_incl_pop,
    data_sub_dir="netcdf-scm-crunched",
    force=False,
    small_number_workers=60,
    small_threshold=100,
    medium_number_workers=20,
    medium_threshold=400,
    force_lazy_threshold=1000,
    cell_weights=None,
)

In [ ]:
checker = "./country-crunch-country-weighted/netcdf-scm-crunched/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/ssp245/r3i1p1f1/Amon/tas/gn/v20191203/netcdf-scm_tas_Amon_ACCESS-ESM1-5_ssp245_r3i1p1f1_gn_201501-210012.nc"
checker = netcdf_scm.io.load_scmrun(checker)
checker["popn_weighted"] = checker["region"].str.contains("Popn")
checker["geographic_region"] = checker["region"].str.replace("Popn weighted ", "")

ax = plt.figure(figsize=(12, 8)).add_subplot(111)
checker.time_mean("AC").lineplot(hue="geographic_region", style="popn_weighted", ax=ax)

In [ ]:
!find /data/cmip6/CMIP6/CMIP -name 'KIOST-ESM' -type d

In [ ]:
!ls /data/cmip6/CMIP6/ScenarioMIP/NCAR/CESM2

In [ ]:
!find /data/cmip6/CMIP6/CMIP/KIOST/KIOST-ESM -type f -name 'tas_Amon*'

In [ ]:
!find /data/cmip6/CMIP6/ScenarioMIP -name 'KIOST-ESM' -type d

In [ ]:
!find /data/cmip6/CMIP6/ScenarioMIP/KIOST/KIOST-ESM -type f -name 'tas_Amon*_ssp245_*'

In [ ]:
!find /data/cmip6/CMIP6/ScenarioMIP/CCCR-IITM/IITM-ESM -type f -name '*_ssp245_*'

In [ ]:
!find {CRUNCH_DIR} -name '*_ssp245_*' -type f  #| wc -l